### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

/Users/pshekarappa/pravn27/eng-it-career/full-stack-data-ai/rag/rag-learning-journey/krish-naik/course-2/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Read all PDF files from the data/pdf_files directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: agetic-ai-build.pdf
  ✓ Loaded 68 pages

Processing: Ebook-Agentic-AI.pdf
  ✓ Loaded 60 pages

Processing: The-Agentic-AI.pdf
  ✓ Loaded 21 pages

Total documents loaded: 149


In [10]:
all_pdf_documents

[Document(metadata={'producer': 'Adobe PDF Library', 'creator': 'PyPDF', 'creationdate': '2025-10-28T10:07:16+05:30', 'author': 'Sinan Ozdemir', 'moddate': '2025-10-30T16:30:06+05:30', 'title': 'Building Agentic AI: Workflows, Fine-Tuning, Optimization, and Deployment', 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States', 'codemantra, llc': 'http://www.codemantra.com', 'source': '../data/pdf_files/agetic-ai-build.pdf', 'total_pages': 68, 'page': 0, 'page_label': 'Cover', 'source_file': 'agetic-ai-build.pdf', 'file_type': 'pdf'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library', 'creator': 'PyPDF', 'creationdate': '2025-10-28T10:07:16+05:30', 'author': 'Sinan Ozdemir', 'moddate': '2025-10-30T16:30:06+05:30', 'title': 'Building Agentic AI: Workflows, Fine-Tuning, Optimization, and Deployment', 'universal pdf': 'The process that creates this PDF constitutes a tra

In [5]:
# Text splitting get into Chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [6]:
chunked_pdf_documents = split_documents(all_pdf_documents)
chunked_pdf_documents

Split 149 documents into 326 chunks

Example chunk:
Content: Building Agentic AI...
Metadata: {'producer': 'Adobe PDF Library', 'creator': 'PyPDF', 'creationdate': '2025-10-28T10:07:16+05:30', 'author': 'Sinan Ozdemir', 'moddate': '2025-10-30T16:30:06+05:30', 'title': 'Building Agentic AI: Workflows, Fine-Tuning, Optimization, and Deployment', 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States', 'codemantra, llc': 'http://www.codemantra.com', 'source': '../data/pdf_files/agetic-ai-build.pdf', 'total_pages': 68, 'page': 1, 'page_label': 'i', 'source_file': 'agetic-ai-build.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Adobe PDF Library', 'creator': 'PyPDF', 'creationdate': '2025-10-28T10:07:16+05:30', 'author': 'Sinan Ozdemir', 'moddate': '2025-10-30T16:30:06+05:30', 'title': 'Building Agentic AI: Workflows, Fine-Tuning, Optimization, and Deployment', 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States', 'codemantra, llc': 'http://www.codemantra.com', 'source': '../data/pdf_files/agetic-ai-build.pdf', 'total_pages': 68, 'page': 1, 'page_label': 'i', 'source_file': 'agetic-ai-build.pdf', 'file_type': 'pdf'}, page_content='Building Agentic AI'),
 Document(metadata={'producer': 'Adobe PDF Library', 'creator': 'PyPDF', 'creationdate': '2025-10-28T10:07:16+05:30', 'author': 'Sinan Ozdemir', 'moddate': '2025-10-30T16:30:06+05:30', 'title': 'Building Agentic AI: Workflows, Fine-Tuning, Optimization, and Deployment', 'universal pdf': 'The process that creates this PDF co

### Embedding And vectorStoreDB

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

# initialize the embedding manager
embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [9]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 326


In [10]:
# Convert the text to embeddings
texts = [doc.page_content for doc in chunked_pdf_documents]

# Generate the Embeddings
embeddings = embedding_manager.generate_embeddings(texts)

# store in the vector database
vectorstore.add_documents(chunked_pdf_documents, embeddings)

Generating embeddings for 326 texts...


Batches: 100%|██████████| 11/11 [00:01<00:00,  7.54it/s]


Generated embeddings with shape: (326, 384)
Adding 326 documents to vector store...
Successfully added 326 documents to vector store
Total documents in collection: 652


### Retriever Pipeline From VectorStore

In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever = RAGRetriever(vectorstore, embedding_manager)

In [12]:
rag_retriever.retrieve("Creating the first agent in LangGraph")

Retrieving documents for query: 'Creating the first agent in LangGraph'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.10it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_e6707f31_42',
  'content': '89Case Study 3: From RAG to Agents\nListing 4.1  Creating the first agent in LangGraph\nfrom os import getenv\n# Initialize the language model\nllm = ChatOpenAI(\n model="openai/gpt-4.1-mini",\n temperature=0,\n base_url="https://openrouter.ai/api/v1",\n api_key=getenv("OPENROUTER_API_KEY"),\n extra_body={\n           "usage": {"include": True}\n       }\n )\n# Define the tools for the agent\ntools = [look_up_evidence, run_sql_against_database, get_database_schema]\n# Create the ReAct agent using LangGraph\'s create_react_agent\ncheckpointer = MemorySaver()  # For conversation memory\nreact_agent = create_react_agent(\n   model=llm,\n   tools=tools,\n   checkpointer=checkpointer,\n   prompt="""You are a helpful SQL assistant. You can: ....”””\nIf you’re curious about what this agent looks like in LangGraph, Figure 4.2 shows the \nnodes that are automatically set up by the create_react_agent predefined function. \nThe “agent” node will invoke a L

### Integration Vectordb Context pipeline With LLM output

In [22]:
# Simple RAG pipeline with OpenAI LLM
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

# 1. Initialize the LLM
open_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(api_key=open_api_key, model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=500)

# 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    # retriever the context
    results = retriever.retrieve(query,top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    # generate the answer using OPENAI LLM
    prompt = f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response = llm.invoke(prompt)
    return response.content


In [23]:
answer = rag_simple("Creating the first agent in LangGraph", rag_retriever, llm)
print(answer)

Retrieving documents for query: 'Creating the first agent in LangGraph'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.76it/s]


Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
The first agent in LangGraph is created by initializing the language model, defining tools for the agent, and creating the ReAct agent using LangGraph's create_react_agent function.


### Enhanced RAG Pipeline Features

In [28]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke(prompt)
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is Tool definitions for the SQL agent ?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'What is Tool definitions for the SQL agent ?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 18.68it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Answer: Tool definitions for the SQL agent are found on pages 89-95.
Sources: [{'source': 'agetic-ai-build.pdf', 'page': 66, 'score': 0.21159422397613525, 'preview': 'for computer use LLM, 221–222\nevaluating agent tool use, 130–133\nhandoff tools, 144–145\nintegrated for context engineering, 197\nMCP for flexible discovery of, 106\nMoondream coding agent tool, 187–188\npositional bias in selecting, 156\nfor SQL generation agent, 89–95\nstructured output, 23\nsupervisor (...'}, {'source': 'agetic-ai-build.pdf', 'page': 66, 'score': 0.21159422397613525, 'preview': 'for computer use LLM, 221–222\nevaluating agent tool use, 130–133\nhandoff tools, 144–145\nintegrated for context engineering, 197\nMCP for flexible discovery of, 106\nMoondream coding agent tool, 187–188\npositional bias in selecting, 156\nfor SQL generation agent, 89–95\nstructured output, 23\nsupervisor (...'}, {'source': 'agetic-ai-build.pdf', 'page': 66, 'score': 0.14330685138702393, 'preview': 'throughput, 239–241, 268\

In [29]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke(prompt)
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("Tool definitions for the SQL agent", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])


Retrieving documents for query: 'Tool definitions for the SQL agent'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.18it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
for computer use LLM, 221–222
evaluating agent tool use, 130–133
handoff tools, 144–145
integrated for context engineering, 197
MCP for flexible discovery of, 106
Moondream coding agent tool, 187–188
positional bias in selecting, 156
for SQL generatio

n agent, 89–95
structured output, 23
supervisor (tool-calling) architecture, 141
testing agentic tool use, 149–156
top_k, 17 , 19, 1 14, 235
top_p, 18, 19
traceability, 1 19–121
training
adding knowledge after initial, 252–253
for distillation, 268
for embedding models, 281–283
to fine-tune generative models, 246–247
performance/reliability via, 12
of reasoning models, 204
See also learning

for computer use LLM, 221–222
evaluating agent tool use, 130–133
handoff tools, 144–145
integrated for context engineering, 197
MCP for flexible discovery of, 106
Moondream coding agent tool, 187–188
positional bias in selecting, 156
for SQL generation agent, 89–95
structured output, 23
supervisor (tool-calling) architecture, 141
testing agentic tool use, 149–156
top_k, 17 , 19, 1 14, 235
top_p, 18, 19
traceability, 1 19–121
training
adding knowledge after initial, 252–253
for distillation, 268
for embedding models, 281–283
to fine-tune generative models, 246–247
performance/reliability via, 12
of 